## Import Data

In [24]:
import numpy as np
import imdb 
import pandas as pd
from error_generator import Explicit_Missing_Value
from error_generator import Implicit_Missing_Value
from error_generator import White_Noise
from error_generator import Gaussian_Noise
from error_generator import Random_Active_Domain
from error_generator import Similar_Based_Active_Domain
from error_generator import Typo_Keyboard
from error_generator import Typo_Butterfingers
from error_generator import Word2vec_Nearest_Neighbor
from error_generator import Value_Selector
from error_generator import List_selected
from error_generator import Read_Write
from error_generator import Error_Generator
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

anime_data = pd.read_csv("new_anime_data1.csv")

anime_data['episodes'] = anime_data['episodes'].replace('Unknown', np.nan)


## Cleaning

In [25]:
l1 = []
l2 = []
l3 = []


anime_data['genre'].fillna('',  inplace=True)
anime_data['overview'].fillna('',  inplace=True)
anime_data['type'].fillna('',  inplace=True)
for index, row in anime_data.iterrows():
    item = row['genre']
    if(pd.isnull(item)):
            item =""
    else:
        if isinstance(item, (list, tuple)):
            item = ','.join(item)
        else:
            item = item.replace(" ","")
            item = item.replace("[","")
            item = item.replace("]","")
            item = item.replace("'","")
    l1.append(item) 
      

for index, row in anime_data.iterrows():
    item = row['overview']
    if(pd.isnull(row['overview'])):
            item = ""
    else:
       # for item in anime_data['overview']:
            if isinstance(item, (list, tuple)):
                item = ','.join(item)
            else:
                item = item.replace("[","")
                item = item.replace("]","")
    l2.append(item) 

for index, row in anime_data.iterrows():
    item = row['type']
    if(pd.isnull(row['type'])):
        item = np.nan
    else:    
        if "movie" in item:
            item = "movie"
        else:
            item = "tv series"
    l3.append(item)   
    
    
anime_data['genre'] = l1   
anime_data['overview'] = l2
anime_data['type'] = l3

#drop dublicate
anime_data.drop_duplicates(inplace = True)



## Sampling

In [26]:
anime_train, anime_test = train_test_split(anime_data, test_size=0.2)

## Generating Errors

In [27]:
def error_generater(dataset, method, n, ignored_columns):

    myselector=List_selected()

    mygen=Error_Generator()

    new_dataset=mygen.error_generator(method_gen=method,selector=myselector,percentage=n,dataset=dataset,mute_column = ignored_columns)
    
    return new_dataset

In [28]:
# generate error in the training set
new_dataset = error_generater(anime_train.values.tolist(), Random_Active_Domain(),30, ignored_columns = [0,1,5,6,7])
Read_Write.write_csv_dataset("./{}.csv".format(Random_Active_Domain().name), new_dataset)
anime_train = pd.read_csv("./{}.csv".format(Random_Active_Domain().name), 
                            names = ['Unnamed: 0', 'anime_id', 'name','genre','type', 'episodes','rating', 'members', 'overview'])

---------Change according to random_active_domain method ---------------

row: 573 col: 8 : '"A young boy named Takuto arrives at the island of Southern Cross, and begins to investigate the island's deep-set secrets, to which he is somehow connected.::Anonymous", 'On a cross-shaped island in the South, a mysterious and gigantic humanoid robot known as the "Cybuddy" is discovered sealed in the underground ruins for untold ages. A group known as the "Order of the Glittering Star Cross" attempts to remove the seal by abducting a girl named Wako Agemaki, who they dubbed the "shrine maiden" and is the key to activating the robot. A boy named Takuto Tsunashi arrives with his own Cybuddy named Tauburn to rescue Wako. Thus begins a struggle to unlock the secrets of the advanced technology and prevent disaster.::ANN'' changed to ''  
row: 651 col: 3 : 'Kids,Shoujo,Supernatural' changed to 'Action,Comedy,Mecha,Sci-Fi'  
row: 770 col: 3 : 'Action,Drama,MartialArts,Shounen,SuperPower' changed to '

## Transformation

In [29]:
import nltk
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
def get_words(x):
    bagofwords=[]
    for i in x:
        if i[1]=='NN':
            bagofwords.append(i[0])
        elif i[1]=='NNS':
            bagofwords.append(i[0])
        elif i[1]=='NNP':
            bagofwords.append(i[0])
        elif i[1]=='NNPS':
            bagofwords.append(i[0])
        elif i[1]=='JJ':
            bagofwords.append(i[0])
        elif i[1]=='JJR':
            bagofwords.append(i[0])
        elif i[1]=='JJS':
            bagofwords.append(i[0])
        elif i[1]=='RB':
            bagofwords.append(i[0])
        elif i[1]=='RBR':
            bagofwords.append(i[0])
        elif i[1]=='RBS':
            bagofwords.append(i[0])
    return bagofwords

def clean_words(x):
    b=nltk.pos_tag(nltk.word_tokenize(x))
    result=get_words(b)
    return result

In [30]:
def feature_transformation(dataset):
    #Hot_Encoding
    df = dataset.genre.str.get_dummies(',')
    
    type_lb = LabelBinarizer()
    X = type_lb.fit_transform(dataset.type.values)

    dfOneHot = pd.DataFrame(X, columns = ["movie/TVseries" for i in range(X.shape[1])])
    dataset = pd.concat([dataset, dfOneHot], axis=1)
    dataset = pd.concat([dataset, df], axis=1)

    dataset['movie/TVseries'].fillna(0, inplace=True)
    
    #Bag of Words
    summary_doc = dataset['overview'].fillna("").map(clean_words)
    summary_doc =summary_doc.apply(','.join)
 
    vectorizer = TfidfVectorizer()
    overview_feature = vectorizer.fit_transform(summary_doc).toarray()

    df = pd.DataFrame(overview_feature, columns = ["word"+ str(int(i)) for i in range(overview_feature.shape[1])])
    dataset = pd.concat([dataset, df], axis=1)
    
    dataset = dataset.drop(columns=['Unnamed: 0', 'anime_id', 'name', 'genre', 'overview', 'type'])
    
    #drop NaN values
    dataset = dataset.dropna()

    return dataset



In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
anime_train = feature_transformation(anime_train)
anime_test = feature_transformation(anime_test)

In [32]:
anime_train.shape

(1553, 6836)

In [33]:
anime_test.shape

(143, 3559)

In [34]:
anime_train_columns = list(anime_train)
anime_test_columns = list(anime_test)

for col_name in anime_train_columns:
    if col_name not in anime_test_columns:
        anime_test[str(col_name)] = 0
        
for col_name in anime_test_columns:
    if col_name not in anime_train_columns:
        anime_test.drop([str(col_name)], axis=1, inplace=True)
        

In [35]:
#anime_train = anime_train.dropna()
print(anime_train.shape)
print(anime_test.shape)

(1553, 6836)
(143, 6836)


In [36]:
anime_y_train = anime_train['rating']
anime_X_train = anime_train.drop(columns=['rating'])

anime_y_test = anime_test['rating']
anime_X_test = anime_test.drop(columns=['rating'])


## Feature Selection

In [37]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

selector = SelectKBest(score_func=f_regression,k=700)#anime_X_test.shape[1]-1)
features = selector.fit(anime_X_train, anime_y_train)

# summarize scores
np.set_printoptions(precision=3)
#print(fit.scores_)

print(anime_X_train.shape)
print(anime_X_test.shape)
anime_X_train = features.transform(anime_X_train)
anime_X_test = features.transform(anime_X_test)
print(anime_X_train.shape)
print(anime_X_test.shape)

(1553, 6835)
(143, 6835)
(1553, 700)
(143, 700)


c:\users\master\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\feature_selection\univariate_selection.py:299: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
c:\users\master\appdata\local\programs\python\python36-32\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
c:\users\master\appdata\local\programs\python\python36-32\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
c:\users\master\appdata\local\programs\python\python36-32\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


## Modeling

In [38]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(anime_X_train, anime_y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

## Testing & Evaluation

In [39]:
# Make predictions using the testing set
anime_y_pred = regr.predict(anime_X_test)

# The mean absolute error
print("Mean absolute error: %.2f" % np.sqrt(mean_absolute_error(anime_y_test, anime_y_pred)))

# The mean squared error
print("Mean squared error: %.2f" % np.sqrt(mean_squared_error(anime_y_test, anime_y_pred)))

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(anime_y_test, anime_y_pred))


Mean absolute error: 1.00
Mean squared error: 1.44
Variance score: -1.38
